In [55]:
import os
from tensorflow.python.platform import gfile
import scipy.io.wavfile as wav
from python_speech_features import *
import numpy as np
from subprocess import call
from struct import unpack, pack
from tqdm import tqdm as tqdm

DATA_DIR = os.path.join("/n/sd7/trung/csp", "data", "vivos")

In [86]:
mode = "train"

wset = set()
with open(os.path.join(DATA_DIR, mode, "prompts.txt"), encoding="utf-8") as f:
    for s in f.read().split('\n'):
        s = s.replace(':', '')
        words = s.split(' ')[1:]
        for word in words: 
            if word != '': wset.add(word.lower())

# word-unit
with open(os.path.join(DATA_DIR, "vocab_words.txt"), "w", encoding="utf-8") as f:
    f.write("<oov>\n")
    f.write("\n".join(["%s" % (word) for _, word in
        enumerate(list(wset)) if word != ""]))

# char-unit
cset = set()
for word in wset: cset |= set(word)
with open(os.path.join(DATA_DIR, "vocab_chars.txt"), "w", encoding="utf-8") as f:
    f.write("_\n")
    f.write("\n".join(["%s" % (c) for _, c in
        enumerate(list(cset)) if c != ""]))

vocab_word = { word: i for i, word in enumerate(open(os.path.join(DATA_DIR, "vocab_words.txt")).read().split('\n')) }
vocab_char = { word: i for i, word in enumerate(open(os.path.join(DATA_DIR, "vocab_chars.txt")).read().split('\n')) }

print("Word count:", len(vocab_word))
print("Char count:", len(vocab_char))
print(vocab_char.keys())

Word count: 4861
Char count: 91
dict_keys(['_', 'ầ', 'ự', 'ẽ', 'ẫ', 'ố', 'é', 'ý', 'c', 'á', 'y', 'i', 'à', 'x', 'ú', 'ằ', 'ũ', 'ể', 'ớ', 'ử', 'ư', 'ơ', 'ỉ', 'õ', 'ẵ', 'ó', 'ả', 'ĩ', 'n', 'o', 'ỏ', 'ù', 'ộ', 'u', 'ữ', 'ệ', 'ỵ', 'ê', 'ã', 's', 'đ', 'ứ', 'ủ', 'ă', 'ẩ', 'ỹ', 'ụ', 'a', 'ì', 'e', 'm', 'b', 'ạ', 'd', 'ỡ', 'l', 'ẻ', 'ờ', 'k', 'ợ', '4', 'ị', 'ỷ', 'ẳ', 'p', 'ồ', 'ễ', 'ở', 'ẹ', 'í', 'â', 'h', 'ấ', 'ắ', 'ỳ', 'v', 'ô', 'g', 'ọ', 'ế', 'ổ', 'è', 'q', 'ề', 't', 'ậ', 'ò', 'r', 'ỗ', 'ừ', 'ặ'])


In [70]:
# get mean
mean = np.array([0] * 120)
var = np.array([0] * 120)
count = 0

mode = "train"
with open(os.path.join(DATA_DIR, mode, "prompts.txt"), encoding="utf-8") as f:
    lines = f.read().split("\n")
    for i, s in tqdm(list(enumerate(lines)), desc=mode):
        filename = s.split(' ')[0]
        if filename == "": continue
        wav_filename = os.path.join(DATA_DIR, mode, "waves", filename.split('_')[0], filename + ".wav")
        npy_filename = os.path.join(DATA_DIR, mode, "npy", filename.split('_')[0], filename + ".npy")

        filename = os.path.join(DATA_DIR, mode, "feature", filename + ".htk")
        call([
            "/n/sd7/trung/bin/htk/HTKTools/HCopy",
            wav_filename,
            filename,
            "-C", "/n/sd7/trung/config.lmfb.40ch"
        ])
        fh = open(filename, "rb")
        spam = fh.read(12)
        nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
        veclen = int(sampSize / 4)
        fh.seek(12, 0)
        dat = np.fromfile(fh, dtype=np.float32)
        dat = dat.reshape(len(dat) // veclen, veclen)
        dat = dat.byteswap()
        fh.close()

        for k in range(len(dat)):
            updated_mean = (mean * count + dat[k]) / (count + 1)
            var = (count * var + (dat[k] - mean) * (dat[k] - updated_mean)) / (count + 1)
            mean = updated_mean
            count += 1

train: 100%|██████████| 11661/11661 [09:16<00:00, 20.94it/s]


In [71]:
print(var)

[1.38207185 2.46174385 3.91958839 4.37270776 4.81470204 4.82823999
 4.66911868 4.72899695 4.84014002 4.89239347 4.64227326 4.89814603
 4.8223888  4.44516746 4.2941552  4.08150563 3.86376774 3.77136156
 3.64902139 3.47856072 3.59918557 3.59006455 3.62087033 3.67942348
 3.46131254 3.65319249 3.64677982 3.62159319 3.67318854 3.72135487
 3.64235773 3.35510406 2.90820098 2.40895137 2.19363585 2.17407766
 2.02689838 1.96658986 2.04127664 2.07970579 0.05948409 0.06667083
 0.0807493  0.09019495 0.10737468 0.12435774 0.13387861 0.13918603
 0.14520674 0.15264976 0.15092295 0.14648077 0.14080602 0.13070525
 0.12424339 0.11949739 0.11472416 0.11344693 0.11198919 0.10951017
 0.1091674  0.10774977 0.10555033 0.10365875 0.0996435  0.10382715
 0.10900165 0.11181899 0.11458519 0.11524595 0.11400782 0.10877169
 0.09797947 0.08491204 0.0792967  0.08007722 0.07654737 0.07570489
 0.07636243 0.0745485  0.01118224 0.01146189 0.01261451 0.01341283
 0.01589817 0.01920197 0.0211048  0.0215637  0.02192232 0.0225

In [91]:
outputs = {'train': [], 'test': []}

for mode in ["test", "train"]:
    with open(os.path.join(DATA_DIR, mode, "prompts.txt"), encoding="utf-8") as f:
        lines = f.read().split("\n")
        for i, s in tqdm(list(enumerate(lines)), desc=mode):
            filename = s.split(' ')[0]
            if filename == "": continue
            wav_filename = os.path.join(DATA_DIR, mode, "waves", filename.split('_')[0], filename + ".wav")
            npy_filename = os.path.join(DATA_DIR, mode, "npy", filename + ".npy")

            if True:
                # (rate, sig) = wav.read(wav_filename)
                htk_filename = os.path.join(DATA_DIR, mode, "feature", filename + ".htk")
                fh = open(htk_filename, "rb")
                spam = fh.read(12)
                nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
                veclen = int(sampSize / 4)
                fh.seek(12, 0)
                dat = np.fromfile(fh, dtype=np.float32)
                dat = dat.reshape(len(dat) // veclen, veclen)
                dat = dat.byteswap()
                fh.close()

                dat = (dat - mean) / np.sqrt(var)
                np.save(npy_filename, dat)

            trans = s.lower().split(' ', 1)[1].replace(' ', '_').replace(':', '')
            outputs[mode].append(dict(
                filename=npy_filename,
                target_word=' '.join([str(vocab_word[w.lower()]) if w in words else '0' \
                    for w in trans.split(' ')]),
                target_char=' '.join([str(vocab_char[c.lower()]) for c in trans]),
                trans_words=' '.join(s.lower().split(' ')[1:])
            ))

train: 100%|██████████| 11661/11661 [05:35<00:00, 34.76it/s]


In [96]:
for mode in ["test", "train"]:
    for unit in ["word", "char"]:
        with open(os.path.join(DATA_DIR, "%s_%s" % (unit, mode) + '.csv'), 'w') as f:
            f.write('\t'.join(['sound', 'target', 'trans']) + '\n')
            for o in outputs[mode]:
                f.write('\t'.join([
                    o['filename'],
                    o['target_%s' % unit],
                    o['trans_words']
                ]) + '\n')

In [6]:
mean = np.load(os.path.join(DATA_DIR, "mean.npy"))
var = np.load(os.path.join(DATA_DIR, "var.npy"))

def get_features_npy(infile, outfile):
    fh = open(infile, "rb")
    spam = fh.read(12)
    nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
    veclen = int(sampSize / 4)
    fh.seek(12, 0)
    dat = np.fromfile(fh, dtype=np.float32)
    dat = dat.reshape(len(dat) // veclen, veclen)
    dat = dat.byteswap()
    fh.close()
    
    dat = (dat - mean) / np.sqrt(var)
    # print(len(dat), dat)
    np.save(outfile, dat)